<a href="https://colab.research.google.com/github/zain501/colabneural/blob/main/Draft_neural_group_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# import data
data = pd.read_excel('cleaned_final.xlsx')
data

,loan_amnt,int_rate,installment,annual_inc,zip_code,dti,delinq_2yrs,open_acc,revol_util,total_pymnt,total_rec_prncp,total_rec_late_fee,recoveries,last_pymnt_amnt,acc_now_delinq,loan_is_bad,grade_encoded,home_ownership_encoded
0,11200,6.62,343.89,108000.0,750xx,12.52,0,9,0.662,11676.060000,10504.53,0.0,0.00,343.89,0,0,1,1
1,10000,11.14,328.06,65000.0,085xx,9.58,0,9,0.742,4620.340000,2710.84,0.0,1011.90,328.06,0,1,2,2
2,8000,16.29,282.41,35000.0,440xx,27.84,0,12,0.720,9601.940000,7446.79,0.0,0.00,282.41,0,0,3,3
3,16000,7.90,500.65,110000.0,060xx,28.87,0,21,0.752,16768.480000,16000.00,0.0,0.00,13269.44,0,0,1,2
4,4000,7.90,125.17,155000.0,816xx,17.87,0,7,0.955,4252.060000,3749.42,0.0,0.00,125.17,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49877,22000,13.99,751.81,94000.0,070xx,15.38,0,14,0.759,4155.170000,2015.61,0.0,1148.85,751.81,0,1,3,3
49878,4000,8.90,127.02,35360.0,934xx,21.14,0,8,0.550,4253.460000,3619.43,0.0,68.31,127.02,0,1,1,3
49879,27050,18.25,981.32,78000.0,996xx,19.42,0,9,0.889,35220.292295,27050.00,0.0,0.00,4807.35,0,0,4,2
49880,9600,7.90,300.39,45000.0,448xx,19.52,0,16,0.510,10812.620905,9600.00,0.0,0.00,307.16,0,0,1,2


In [ ]:
data['zip_code'] = data['zip_code'].str.extract(r'(\d+)').astype(int)
data['zip_code']

In [13]:
features = data.drop(columns=['loan_is_bad']).values
labels = data['loan_is_bad'].values

tensors = tf.data.Dataset.from_tensor_slices((features, labels))

In [14]:
train_data = tensors.take(round(len(data)*0.8),0)
test_data = tensors.skip(round(len(data)*0.8),0)
len(train_data)

39906

batch = 32, 64, 128. If computational resources allow, larger batch sizes can be chosen to speed up training. However, be mindful of selecting batch sizes that are too large, as they may lead to memory-related issues.

In [15]:
train_batch = train_data.batch(128)
features, labels = next(iter(train_batch)) # iterate through each batch at training time

In [16]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(5, activation=tf.nn.relu, input_shape=(17, )),  # 17 features
  tf.keras.layers.Dense(5, activation=tf.nn.relu),
  tf.keras.layers.Dense(1)
])


In [ ]:
type(data)

In [30]:
# compile the model
# Convert labels to float32
labels_float32 = tf.keras.backend.cast(labels, dtype='float32')

from keras import backend as K

# Define False Negatives metric
def false_negatives(y_true, y_pred):
    # Calculate false negatives
    fn = K.sum(K.cast(K.greater(y_true, y_pred), dtype='float32'))
    return fn

# Compile model with the custom metrics
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), false_negatives])




epochs = 20~100. In general, a larger number of epochs may lead to better model performance as the model can learn more information from the data. However, too many epochs may also result in overfitting, where the model performs well on the training data but poorly on unseen data. Therefore, it is advisable to gradually increase the number of epochs and observe changes in the model's performance on a validation set.

In [31]:
model.fit(features, labels_float32, epochs=1500)

Epoch 1/1500
4/4 [==============================] - 2s 6ms/step - loss: nan - accuracy: 0.1875 - precision_2: 0.1545 - recall_2: 1.0000 - false_negatives: 1.2500
Epoch 2/1500
4/4 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.1875 - precision_2: 0.1545 - recall_2: 1.0000 - false_negatives: 1.2500
Epoch 3/1500
4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.1875 - precision_2: 0.1545 - recall_2: 1.0000 - false_negatives: 1.2500
Epoch 4/1500
4/4 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.1875 - precision_2: 0.1545 - recall_2: 1.0000 - false_negatives: 1.2500
Epoch 5/1500
4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.1875 - precision_2: 0.1545 - recall_2: 1.0000 - false_negatives: 1.2500
Epoch 6/1500
4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.1875 - precision_2: 0.1545 - recall_2: 1.0000 - false_negatives: 1.2500
Epoch 7/1500
4/4 [==========

In [32]:
# Evaluate the model on the test batch
evaluation = model.evaluate(test_features, test_labels)

# Extract and print the metrics
accuracy = evaluation[1]
precision = evaluation[2]
recall = evaluation[3]
false_negatives = evaluation[4]

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'False Negatives: {false_negatives:.4f}')


4/4 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.1953 - precision_2: 0.1417 - recall_2: 1.0000 - false_negatives: 2.0000
Accuracy: 0.1953
Precision: 0.1417
Recall: 1.0000
False Negatives: 2.0000


In [34]:
# Get predictions on the test batch
predictions = model.predict(test_features)


predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_labels

for i in range(min(10, len(test_features))):  # Displaying predictions for the first 10 samples
    print(f"Sample {i + 1}:")
    print(f"Predicted class: {predicted_classes[i]}, Actual class: {true_classes[i]}")
    print(f"Model confidence: {predictions[i][predicted_classes[i]]:.4f}")
    print("-" * 30)


4/4 [==============================] - 0s 2ms/step
Sample 1:
Predicted class: 0, Actual class: 0
Model confidence: 2184.5918
------------------------------
Sample 2:
Predicted class: 0, Actual class: 0
Model confidence: 3875.7004
------------------------------
Sample 3:
Predicted class: 0, Actual class: 1
Model confidence: 2630.2920
------------------------------
Sample 4:
Predicted class: 0, Actual class: 0
Model confidence: 1398.3402
------------------------------
Sample 5:
Predicted class: 0, Actual class: 0
Model confidence: 654.9946
------------------------------
Sample 6:
Predicted class: 0, Actual class: 0
Model confidence: 624.1678
------------------------------
Sample 7:
Predicted class: 0, Actual class: 1
Model confidence: 4256.5684
------------------------------
Sample 8:
Predicted class: 0, Actual class: 0
Model confidence: 2541.0100
------------------------------
Sample 9:
Predicted class: 0, Actual class: 0
Model confidence: 2749.8762
------------------------------
Sample